Need to comment this file since we're gonna be working on notebooks

## Imports

In [1]:
import pysindy as ps
from pysindy import PDELibrary
from pysindy.optimizers import STLSQ
from pysindy.differentiation import SpectralDerivative
import matplotlib.pyplot as plt
from PIL import Image
import os
import numpy as np

## Data Selection

In [2]:
""" Features directory of the changemaps """
#features_dir = r"../../generations/changemap_generations/cropped_cropped/result_20_cut_1_png"
#features_dir = r"../../generations/changemap_generations/cropped_resized/result_20_cut_1_png"

""" Features directory of the generated registered by deformation (these images are .jpg)"""
#features_dir = r"C:\Users\ariel\PycharmProjects\MLDM_Project\generations\deformation_generations\registered\24"
#features_dir = r"../../generations/deformation_generations/deformations/24"

""" Features directory of the phase correlation-matched images """
features_dir = r"C:\Users\ariel\PycharmProjects\MLDM_Project\generations\phase_correlated_generations\patch_match_20_2"

""" Resolution (if necessary) to resize """
res = 500

In [3]:
frames = []

filenames = sorted(os.listdir(features_dir))
print(filenames) # Checking they've been imported in the correct order
for filename in filenames:
    if filename.endswith(".png"):
        img = Image.open(os.path.join(features_dir, filename))
        width, height = img.size
        #img = img.resize((res, res))
        img = img.convert('L')
        img = np.asarray(img)
        img = img / 255
        frames.append(img)
        
res = np.shape(frames[0])[0]

['aligned_00.png', 'aligned_01.png', 'aligned_02.png', 'aligned_03.png', 'aligned_04.png', 'aligned_05.png', 'aligned_06.png', 'aligned_07.png', 'aligned_08.png', 'aligned_09.png', 'aligned_10.png', 'aligned_11.png', 'aligned_12.png', 'aligned_13.png', 'aligned_14.png', 'aligned_15.png', 'aligned_16.png', 'aligned_17.png', 'aligned_18.png', 'aligned_19.png', 'aligned_20.png', 'aligned_21.png', 'aligned_22.png', 'aligned_23.png', 'aligned_24.png', 'aligned_25.png', 'aligned_26.png', 'aligned_27.png', 'aligned_28.png', 'aligned_29.png', 'aligned_30.png', 'aligned_31.png', 'aligned_32.png', 'aligned_33.png', 'aligned_34.png', 'aligned_35.png', 'aligned_36.png', 'aligned_37.png', 'aligned_38.png', 'aligned_39.png', 'aligned_40.png', 'aligned_41.png', 'aligned_42.png', 'aligned_43.png', 'aligned_44.png', 'aligned_45.png', 'aligned_46.png', 'aligned_47.png', 'aligned_48.png', 'aligned_49.png']


## Reshaping data, creating spatial grid

In [4]:
#Creating spatial grid
features = np.stack(frames, axis=0)
features = np.reshape(features, (len(features), res, res, 1))
spatial_grid_x, spatial_grid_y = np.meshgrid(np.arange(res), np.arange(res))
spatial_grid = np.stack((spatial_grid_x, spatial_grid_y), axis=-1)
print("Spatial grid shape:", np.shape(spatial_grid))
print("Features shape:", np.shape(features))

Spatial grid shape: (500, 500, 2)
Features shape: (50, 500, 500, 1)


# Defining the feature library

In [5]:
lib = ps.PDELibrary(
    library_functions=[
        lambda x:  x,
        lambda x:  x ** 2,
        lambda x:  x ** 3,
    ],
    function_names = [
        lambda x: x,
        lambda x: x + x,
        lambda x: x + x + x,
    ],
    derivative_order=4,
    spatial_grid=spatial_grid,
    include_interaction=False,
    multiindices=[[0,2], [2,0], [4,0], [0,4]],
    differentiation_method=ps.SpectralDerivative,
)

## Defining the optimizer and model

In [6]:
opt = STLSQ(threshold=0.001, alpha=0.1, verbose=True, normalize_columns=False)
model = ps.SINDy(feature_library=lib, optimizer=opt, feature_names=["u"])

## Defininng x_dot

In [7]:
x = np.asarray(features)
dt = np.arange(0, x.shape[0]*0.05, 0.05) #Not necessary as time has been forced on source

x_dot = model.differentiate(x=x) 

'''plt.imshow(x_dot[25,:,:,0], cmap='gray')
plt.colorbar(label='Intensity')
plt.show()'''

print("Shape of x_dot:", np.shape(x_dot))

[50  1  1  1]
Shape of x_dot: (50, 500, 500, 1)


## Fitting the model

In [8]:
print("Fitting model...")
model.fit(x=x, x_dot=x_dot)
model.print()

Fitting model...
[  1   1 500   1]
[  1 500   1   1]
[  1 500   1   1]
[  1   1 500   1]
SLTSQ
n_features_selected =  7
n_features =  7
n_targets =  1
n_samples =  12500000
x.shape =  (12500000, 7)
y.shape =  (12500000, 1)

 Iteration ... |y - Xw|^2 ...  a * |w|_2 ...      |w|_0 ... Total error: |y - Xw|^2 + a * |w|_2
         0 ... 1.4108e+01 ... 2.0172e+01 ...          5 ... 3.4280e+01
         1 ... 1.4108e+01 ... 2.0104e+01 ...          4 ... 3.4212e+01
         2 ... 1.4108e+01 ... 2.0107e+01 ...          4 ... 3.4216e+01
(u)' = -3.389 u + 10.900 uu + -8.416 uuu + -0.001 u_22
